In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender


#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [2]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')
#UCM_all = scipy.sparse.load_npz('./Matrix/UCM_all_matrix.npz')
#UCM_train1 = scipy.sparse.load_npz('UCM_train_matrix1.npz')



#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

In [5]:
UserCBFrecommender = UserKNNCBFRecommender(UCM_train,URM_train) 
UserCBFrecommender.fit(topK=300, shrink=80, feature_weighting = "BM25")
result = UserCBFrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

Similarity column 22143 ( 44 % ), 603.00 column/sec, elapsed time 0.61 min
Similarity column 50446 ( 100 % ), 645.43 column/sec, elapsed time 1.30 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 9.51 seconds. Users per second: 1017
Recommender MAP is= 0.24810987000158252


In [7]:
import pandas as pd
from numpy import *
import numpy as np
from scipy import sparse



data_playlists= pd.read_csv('all/train.csv',low_memory = False)
data_tracks= pd.read_csv('all/tracks.csv',low_memory = False)



#SPLITTAGGIO DEI DATI

#prendo il 20% delle random playlist da target playlist
target_playlist= pd.read_csv('all/target_playlists.csv',low_memory = False)
random_playlist=target_playlist.iloc[5000:10000]
random_id = random_playlist.playlist_id.values
random_data=data_playlists.loc[data_playlists['playlist_id'].isin(random_id)]
#random_data.shape = (114513, 2)
n=round((114513/100)*20)
random_subset = random_data.sample(n=n)

#prendo il 20% delle sequential playlist da target playlist
sequential_playlist=target_playlist.iloc[:5000]
sequential_id = sequential_playlist.playlist_id.values
sequential_data=data_playlists.loc[data_playlists['playlist_id'].isin(sequential_id)]
n1=round((115553/100)*20)
for pid in sequential_id:
    playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
    start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
    end=start+round((playlistID.shape[0]*50)/100)

    dropping=list(range(start, end))    
    sequential_data=sequential_data.drop(dropping)


sequential_subset = sequential_data.sample(n=n1)

In [6]:
indexes=data_playlists.index.values
index_random=random_subset.index.values
index_sequential=sequential_subset.index.values
print(indexes.shape)
print(index_random.shape)
print(index_sequential.shape)

(1211791,)
(22903,)
(23111,)


In [7]:
data_playlists=data_playlists[~data_playlists.isin(sequential_subset)].dropna()
data_playlists=data_playlists[~data_playlists.isin(random_subset)].dropna()

In [11]:
#CREAZIONE DELLA UCM_ALL
UCM_train=zeros((50446,19412))
URM_train=zeros((50446,20635))

#inizializzazione completa
for row in data_playlists.itertuples():
    riga = int(row.playlist_id)
    #print(riga)
    #track=data_tracks.loc[data_tracks['track_id'] == int(row.track_id)]
    #print(colonna)
    colonna= int(row.track_id)
    URM_train[riga,colonna]=1
   #UCM_train[riga,int(track.album_id)]=1
    #UCM_train[riga,12744+int(track.artist_id)]=1

#UCM_train=sparse.csr_matrix(UCM_train)   
#sparse.save_npz('UCM_train_matrix.npz', UCM_train)
URM_train=sparse.csr_matrix(URM_train)   







In [11]:
import pandas as pd
from numpy import *
import numpy as np
from scipy import sparse



data_playlists= pd.read_csv('all/train.csv',low_memory = False)
data_tracks= pd.read_csv('all/tracks.csv',low_memory = False)



#SPLITTAGGIO DEI DATI

#prendo il 20% delle random playlist da target playlist
target_playlist= pd.read_csv('all/target_playlists.csv',low_memory = False)
random_playlist=target_playlist.iloc[5000:10000]
random_id = random_playlist.playlist_id.values
random_data=data_playlists.loc[data_playlists['playlist_id'].isin(random_id)]
#random_data.shape = (114513, 2)
n=round((114513/100)*20)
random_subset = random_data.sample(n=n)

#prendo il 20% delle sequential playlist da target playlist
sequential_playlist=target_playlist.iloc[:5000]
sequential_id = sequential_playlist.playlist_id.values
sequential_data=data_playlists.loc[data_playlists['playlist_id'].isin(sequential_id)]
n1=round((115553/100)*20)
for pid in sequential_id:
    playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
    start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
    end=start+round((playlistID.shape[0]*50)/100)

    dropping=list(range(start, end))    
    sequential_data=sequential_data.drop(dropping)


sequential_subset = sequential_data.sample(n=n1)


#CREAZIONE DELLA UCM_ALL
UCM_train1=zeros((50446,19412))

#inizializzazione completa
for row in data_playlists.itertuples():
    riga = row.playlist_id
    #print(riga)
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    #print(colonna)
    UCM_train1[riga,int(track.album_id.values)]=1
    UCM_train1[riga,12744+int(track.artist_id.values)]=1
    
for row in sequential_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    #print(colonna)
    UCM_train1[riga,int(track.album_id.values)]=0
    UCM_train1[riga,12744+int(track.artist_id.values)]=0
for row in random_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    #print(colonna)
    UCM_train1[riga,int(track.album_id.values)]=0
    UCM_train1[riga,12744+int(track.artist_id.values)]=0

UCM_train1=sparse.csr_matrix(UCM_train1)   
sparse.save_npz('UCM_train_matrix1.npz', UCM_train1)




In [5]:
data_playlists= pd.read_csv('all/train.csv',low_memory = False)
data_tracks= pd.read_csv('all/tracks.csv',low_memory = False)



#SPLITTAGGIO DEI DATI

#prendo il 20% delle random playlist da target playlist
target_playlist= pd.read_csv('all/target_playlists.csv',low_memory = False)
random_playlist=target_playlist.iloc[5000:10000]
random_id = random_playlist.playlist_id.values
random_data=data_playlists.loc[data_playlists['playlist_id'].isin(random_id)]
#random_data.shape = (114513, 2)
n=round((114513/100)*20)
random_subset = random_data.sample(n=n)

#prendo il 20% delle sequential playlist da target playlist
sequential_playlist=target_playlist.iloc[:5000]
sequential_id = sequential_playlist.playlist_id.values
sequential_data=data_playlists.loc[data_playlists['playlist_id'].isin(sequential_id)]
n1=round((115553/100)*20)
for pid in sequential_id:
    playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
    start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
    end=start+round((playlistID.shape[0]*50)/100)

    dropping=list(range(start, end))    
    sequential_data=sequential_data.drop(dropping)


sequential_subset = sequential_data.sample(n=n1)

In [35]:
#inizializzazione completa
for row in sequential_subset.itertuples():
    riga = row.playlist_id
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    colonna1= int(track.album_id.values)
    colonna2=12744+int(track.artist_id.values)
    if UCM_train1[riga,colonna1]==1:
        print("Error")
        print(" Riga = {}, Colonna1 = {}".format(riga, colonna1))  

    if UCM_train1[riga,colonna2]==1:
        print("Error")
        print(" Riga = {}, Colonna2 = {}".format(riga, colonna2))  
        




In [36]:
UCM_train1

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
from tqdm import tqdm

#UCM_train1=UCM_train1.todense()


for row in tqdm(sequential_subset.itertuples()):
    riga = row.playlist_id
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    colonna1= int(track.album_id.values)
    colonna2=12744+int(track.artist_id.values)
    UCM_train1[riga,colonna1]=0

    UCM_train1[riga,colonna2]=0


       

23111it [00:25, 920.55it/s] 


In [39]:
from scipy import sparse


UCM_train1=sparse.csr_matrix(UCM_train1)   



In [40]:
UserCBFrecommender = UserKNNCBFRecommender(UCM_train1,URM_train) 
UserCBFrecommender.fit(topK=300, shrink=80)
result = UserCBFrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

Similarity column 16138 ( 32 % ), 457.20 column/sec, elapsed time 0.59 min
Similarity column 45201 ( 90 % ), 525.42 column/sec, elapsed time 1.43 min
Similarity column 50446 ( 100 % ), 529.45 column/sec, elapsed time 1.59 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 10.65 seconds. Users per second: 909
Recommender MAP is= 0.200830856152805
